In [1]:
import pandas as pd
from pyserini.search.lucene import LuceneSearcher
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
import string

d:\Hafiz\Kuliah\Semester 5\Information Retrieval\Projek\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', 500)

In [3]:
istilah_kesehatan = {
    # Tekanan Darah
    "hipertensi": "tekanan darah tinggi",
    "hipotensi": "tekanan darah rendah",

    # Gula Darah
    "hiperglikemia": "kadar gula darah tinggi",
    "hipoglikemia": "kadar gula darah rendah",
    "diabetes melitus": "penyakit kencing manis",

    # Pernapasan
    "dispnea": "sesak napas",
    "rinorea": "pilek atau ingusan",
    "epistaksis": "mimisan",
    "faringitis": "radang tenggorokan",
    "laringitis": "radang pita suara",
    "asma": "bengek",

    # Pencernaan
    "dispepsia": "mag atau gangguan pencernaan",
    "konstipasi": "sembelit atau susah buang air besar",
    "diare": "mencret",
    "hemoroid": "wasir atau ambeien",
    "apendisitis": "radang usus buntu",
    "gastritis": "radang lambung",

    # Kulit dan Alergi
    "urtikaria": "biduran atau kaligata",
    "dermatitis": "eksim atau radang kulit",
    "varisela": "cacar air",
    "morbili": "campak atau tampek",
    "veruka": "kutil",
    "tinea pedis": "kutu air",
    "lotion": "losion",

    # Kepala dan Saraf
    "sefalgia": "sakit kepala atau pusing",
    "insomnia": "susah tidur",
    "sinkop": "pingsan",
    "konvulsi": "kejang",

    # Umum
    "pireksia": "demam",
    "hipertermia": "suhu tubuh sangat tinggi",
    "hipotermia": "suhu tubuh sangat rendah",
    "mialgia": "nyeri otot",
    "artralgia": "nyeri sendi",
    "fatik": "kelelahan atau rasa capai",
    "edema": "bengkak",
    "pruritus": "gatal-gatal",
    "anemia": "kurang darah",
    "karsinoma": "kanker",
    "neoplasma": "tumor",
    "halitosis": "bau mulut",
    "kalkulus renal": "batu ginjal",
    "moisturizing": "moisturizer"
}

In [4]:
stop_words = set(stopwords.words('indonesian'))

In [5]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk stemming
def stem_tokens(tokens):
    lemmas = [stemmer.stem(token) for token in tokens]
    return lemmas

In [6]:
def normalize_text(text, dictionary):
    tokens = text.split()

    normalized_tokens = []
    for token in tokens:
        normalized_token = dictionary.get(token, token)
        normalized_tokens.append(normalized_token)

    return " ".join(normalized_tokens)

In [7]:
def preprocess_query(text):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = ''.join([char for char in text if char not in string.punctuation])
    text = normalize_text(text, istilah_kesehatan)
    tokens = text.split()
    stemmed_tokens = stem_tokens(tokens)
    
    return ' '.join(stemmed_tokens)

In [8]:
queries = [
  'gejala diabetes',
  'makanan sehat untuk jantung',
  'kesehatan mental',
  'efek samping vaksin covid-19',
  'manfaat buah'
]

In [9]:
queries = [preprocess_query(query) for query in queries]

In [88]:
print(queries)

['gejala diabetes', 'makan sehat jantung', 'sehat mental', 'efek samping vaksin covid19', 'manfaat buah']


# Membaca Dataframe

In [14]:
df = pd.read_csv('../artikel/raw_health_articles.csv')
df.drop(columns=['description', 'content'], inplace=True)
df.head()

,article_index,title,tag,link,date,source
0,1,5 Cara Merawat Ginjal agar Sehat & Cegah Penyakit Ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,3 September 2025,Biofarma
1,2,7 Tips Untuk Menjaga Kesehatan Mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,29 Agustus 2025,Biofarma
2,3,Jangan Sepelekan! 10 Penyebab Campak yang Sering Terabaikan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,28 Agustus 2025,Biofarma
3,4,"Campak - Penyebab, Gejala, dan Pencegahannya",Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,25 Agustus 2025,Biofarma
4,5,Cara Mudah Menjaga Kesehatan Gigi dan Mulut agar Terhindar dari Penyakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,20 Agustus 2025,Biofarma


In [12]:
# Fungsi untuk print hasil
def display_results(query, hits):
    print(f'\n{query}')

    if not hits:
        print("Tidak ada dokumen yang sesuai.")
        return
    
    results_with_scores = []
    
    for i in range(len(hits)):
        matching_row = df[df['article_index'] == int(hits[i].docid)].copy()
        
        if not matching_row.empty:
            # Add score to the dataframe row
            matching_row = matching_row.copy()
            matching_row['score'] = hits[i].score
            matching_row['rank'] = i + 1
            results_with_scores.append(matching_row)
            
            # print(f'{i+1:2} ID: {hits[i].docid:4} Score: {hits[i].score:.5f}')
            # print(f'   Title: {matching_row.iloc[0]["title"]}')
            # print()
    
    if results_with_scores:
        result_df = pd.concat(results_with_scores, ignore_index=True)
        return result_df
    else:
        print("No matching documents found in DataFrame.")
        return pd.DataFrame()

## Bm25 Retrieval (Title)

In [10]:
searcher_title = LuceneSearcher('../indexing/indexes/title_jsonl')
searcher_content_title = LuceneSearcher('../indexing/indexes/title_content_jsonl')
searcher_desc_content = LuceneSearcher('../indexing/indexes/title_desc_content_jsonl')

In [94]:
# Test the function with a query
query = queries[4]  
hits = searcher_title.search(query)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


manfaat buah

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,26,Manfaat dan Khasiat Buah Manggis: Buah Eksotis Penuh Gizi dari Indonesia,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/manfaat-dan-khasiat-buah-manggis-buah-eksotis-penuh-gizi-dari-indonesia,28 Mei 2025,Biofarma,2.865400,1
1,61,Manfaat Buah Rambutan untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/rambutan-buah-unik-yang-kaya-nutrisi-vitamin-dan-mineral,3 Maret 2025,Biofarma,2.774300,2
2,102,4 Manfaat Buah Belimbing untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/4-manfaat-buah-belimbing-untuk-kesehatan,22 Oktober 2024,Biofarma,2.688300,3
3,81,5 Manfaat Buah Sirsak Yang Baik Bagi Tubuh,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-manfaat-buah-sirsak-yang-baik-bagi-tubuh,2 Januari 2025,Biofarma,2.688299,4
4,299,Cara Menghilangkan Jamur di Baju dengan Mudah dan Efektif,KESEHATAN,https://www.alodokter.com/cara-menghilangkan-jamur-di-baju-dengan-mudah-dan-efektif,16 September 2025,alodokter,2.607500,5
5,82,8 Ragam Manfaat Buah Naga Untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/8-ragam-manfaat-buah-naga-untuk-kesehatan,19 Desember 2024,Biofarma,2.607499,6
6,234,Manfaat Vitamin C dalam Buah Blackcurrant untuk Kesehatan Tubuh,"Vitamin C, Diet dan Nutrisi",https://www.halodoc.com/artikel/manfaat-vitamin-c-dalam-buah-blackcurrant-untuk-kesehatan-tubuh,12 September 2025,halodoc,2.531500,7
7,284,Cara Mengeluarkan Dahak pada Bayi yang Bisa Dicoba di Rumah,Batuk,https://www.halodoc.com/artikel/cara-mengeluarkan-dahak-pada-bayi-yang-bisa-dicoba-di-rumah,11 September 2025,halodoc,2.531499,8
8,53,Manfaat Buah Srikaya: Si Manis dengan Segudang Khasiat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/manfaat-buah-srikaya-si-manis-dengan-segudang-khasiat,21 Maret 2025,Biofarma,2.531498,9
9,54,Timun Suri: Buah Segar dengan Beragam Manfaat Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/timun-suri-buah-segar-dengan-beragam-manfaat-kesehatan,18 Maret 2025,Biofarma,2.531497,10


## Bm25 Retrieval (Title + Content)

In [19]:
# Test the function with a query
query = queries[4] 
hits = searcher_content_title.search(query, k=10)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


manfaat buah

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,222,Apa Saja Kandungan Vitamin yang Terdapat dalam Buah Salak?,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandungan-vitamin-yang-terdapat-dalam-buah-salak,15 September 2025,halodoc,2.2503,1
1,299,Cara Menghilangkan Jamur di Baju dengan Mudah dan Efektif,KESEHATAN,https://www.alodokter.com/cara-menghilangkan-jamur-di-baju-dengan-mudah-dan-efektif,16 September 2025,alodokter,2.2434,2
2,301,"Daun Bandotan, Inilah Nutrisi, Manfaat, dan Tips Menggunakannya",KESEHATAN,https://www.alodokter.com/daun-bandotan-inilah-nutrisi-manfaat-dan-tips-menggunakannya,16 September 2025,alodokter,2.2166,3
3,102,4 Manfaat Buah Belimbing untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/4-manfaat-buah-belimbing-untuk-kesehatan,22 Oktober 2024,Biofarma,2.2073,4
4,58,Kenali 5 Manfaat Kurma yang Membantu Menjaga Kesehatan Tubuh,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/kenali-5-manfaat-kurma-yang-membantu-menjaga-kesehatan-tubuh,12 Maret 2025,Biofarma,2.1995,5
5,81,5 Manfaat Buah Sirsak Yang Baik Bagi Tubuh,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-manfaat-buah-sirsak-yang-baik-bagi-tubuh,2 Januari 2025,Biofarma,2.1988,6
6,53,Manfaat Buah Srikaya: Si Manis dengan Segudang Khasiat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/manfaat-buah-srikaya-si-manis-dengan-segudang-khasiat,21 Maret 2025,Biofarma,2.1863,7
7,63,Buah Duku: Superfood Tropis yang Kaya Nutrisi dan Manfaat Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/buah-duku-superfood-tropis-yang-kaya-nutrisi-dan-manfaat-kesehatan,25 Februari 2025,Biofarma,2.1650,8
8,347,6 Manfaat Black Garlic Tunggal dan Cara Mengonsumsinya,HIDUP SEHAT,https://www.alodokter.com/6-manfaat-black-garlic-tunggal-dan-cara-mengonsumsinya,10 September 2025,alodokter,2.1647,9
9,82,8 Ragam Manfaat Buah Naga Untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/8-ragam-manfaat-buah-naga-untuk-kesehatan,19 Desember 2024,Biofarma,2.1592,10


## Bm25 Retrieval (Title + Desc + Content)

In [ ]:
# Test the function with a query
query = queries[3] 
hits = searcher_desc_content.search(query)  

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

print("\nResult DataFrame with scores:")
result_df


manfaat buah

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,222,Apa Saja Kandungan Vitamin yang Terdapat dalam Buah Salak?,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandungan-vitamin-yang-terdapat-dalam-buah-salak,15 September 2025,halodoc,2.2324,1
1,299,Cara Menghilangkan Jamur di Baju dengan Mudah dan Efektif,KESEHATAN,https://www.alodokter.com/cara-menghilangkan-jamur-di-baju-dengan-mudah-dan-efektif,16 September 2025,alodokter,2.2281,2
2,301,"Daun Bandotan, Inilah Nutrisi, Manfaat, dan Tips Menggunakannya",KESEHATAN,https://www.alodokter.com/daun-bandotan-inilah-nutrisi-manfaat-dan-tips-menggunakannya,16 September 2025,alodokter,2.2013,3
3,102,4 Manfaat Buah Belimbing untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/4-manfaat-buah-belimbing-untuk-kesehatan,22 Oktober 2024,Biofarma,2.1961,4
4,81,5 Manfaat Buah Sirsak Yang Baik Bagi Tubuh,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-manfaat-buah-sirsak-yang-baik-bagi-tubuh,2 Januari 2025,Biofarma,2.1946,5
5,58,Kenali 5 Manfaat Kurma yang Membantu Menjaga Kesehatan Tubuh,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/kenali-5-manfaat-kurma-yang-membantu-menjaga-kesehatan-tubuh,12 Maret 2025,Biofarma,2.1860,6
6,53,Manfaat Buah Srikaya: Si Manis dengan Segudang Khasiat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/manfaat-buah-srikaya-si-manis-dengan-segudang-khasiat,21 Maret 2025,Biofarma,2.1779,7
7,82,8 Ragam Manfaat Buah Naga Untuk Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/8-ragam-manfaat-buah-naga-untuk-kesehatan,19 Desember 2024,Biofarma,2.1565,8
8,63,Buah Duku: Superfood Tropis yang Kaya Nutrisi dan Manfaat Kesehatan,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/buah-duku-superfood-tropis-yang-kaya-nutrisi-dan-manfaat-kesehatan,25 Februari 2025,Biofarma,2.1538,9
9,347,6 Manfaat Black Garlic Tunggal dan Cara Mengonsumsinya,HIDUP SEHAT,https://www.alodokter.com/6-manfaat-black-garlic-tunggal-dan-cara-mengonsumsinya,10 September 2025,alodokter,2.1479,10
